In [1]:
import cv2
from tqdm import tqdm
print(cv2.__version__)

4.8.1


In [2]:
cap = cv2.VideoCapture("Videos/Start1.avi")

In [3]:
fps =int(cap.get(cv2.CAP_PROP_FPS))

In [5]:
n = 0
diap = tqdm(iterable=range(0,fps*40,fps))
for i in diap:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(f"Images/frame{int(i/fps)}.jpg",frame)

100%|██████████| 40/40 [00:00<00:00, 108.08it/s]
